## Spark setup

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,415 kB]
Get:13 http://archive.

In [2]:
# Start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis)").getOrCreate()

## Load

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

## Transform


In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
#vine_df.show()

In [13]:
# filter total votes >= 20
vine_t_votes_df = vine_df.filter('total_votes>=20')
#vine_t_votes_df.show()

In [12]:
# filter helpful votes / total votes
helpful_total_votes_df = vine_t_votes_df.filter(vine_t_votes_df['helpful_votes']/vine_t_votes_df['total_votes']>=0.5)
#helpful_total_votes_df.show()

In [24]:
# filter vine Y 
vine_y_df = helpful_total_votes_df.filter(helpful_total_votes_df['vine']=='Y')
vine_y_df.count()

94

In [25]:
# filter vine N
vine_n_df = helpful_total_votes_df.filter(helpful_total_votes_df['vine']=='N')
vine_n_df.count()

40471

In [27]:
# Vine Y (PAID) Analysis
y_total_count = vine_y_df.count()
y_five_stars = vine_y_df.filter(vine_y_df['star_rating']=='5').count()
y_fstar_percentage = 100 * y_five_stars / y_total_count



print(f'Total count of reviews written as part of Vine Paid program: {y_total_count}')
print(f'Total count 5-star reviews in the program: {y_five_stars}')
print(f'The percentage of 5-star reviews for Vinde Paid program is: {y_fstar_percentage:.2f}%')

Total count of reviews written as part of Vine Paid program: 94
Total count 5-star reviews in the program: 48
The percentage of 5-star reviews for Vinde Paid program is: 51.06%


In [28]:
# Vine N (UNPAID) Analysis
n_total_count = vine_n_df.count()
n_five_stars = vine_n_df.filter(vine_n_df['star_rating']=='5').count()
n_fstar_percentage = 100 * n_five_stars / n_total_count



print(f'Total count of reviews not part of the Vine program (unpaid): {n_total_count}')
print(f'Total count 5-star reviews in the program: {n_five_stars}')
print(f'The percentage of 5-star reviews for the unpaid program: {n_fstar_percentage:.2f}%')

Total count of reviews not part of the Vine program (unpaid): 40471
Total count 5-star reviews in the program: 15663
The percentage of 5-star reviews for the unpaid program: 38.70%
